In [ ]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from schemas import StatementOfCashFlows2024  #This could be adjusted through schemas.py
from dotenv import load_dotenv

In [ ]:
PDF_ROOT = "university_pdfs"
OUTPUT_ROOT = "output_cash_flow"
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "54164e43-e77a-4add-89a0-b99a31c1ed87" #Different based on your LLamaCloud account

In [ ]:
load_dotenv() #make sure the API key is in the .env file
extractor = LlamaExtract(project_id = '8c10e62e-3810-4193-915d-d2d11105826d')

#uncomment the below line if you are creating the agent for the first time
# agent = extractor.create_agent(name = "statement_of_cash_flows-2024", data_schema=StatementOfCashFlows2024)

agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
agent.data_schema = StatementOfCashFlows2024
agent.save()

In [ ]:
agent.data_schema

In [ ]:
# Set the path to the final Excel output file
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

# Create a Pandas Excel writer using openpyxl
writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

# Track schools with mismatch between calculated and reported cash change
test = []

# Iterate through all schools (each school is a folder inside PDF_ROOT)
for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    
    # Skip if not a directory
    if not os.path.isdir(school_dir):
        continue

    combined   = {}      # Store combined extracted values for the school
    first_keys = None    # Store the order of extracted keys

    # Loop through PDF files inside the school folder
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue

        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")

        try:
            run  = agent.extract(path)          # Use the agent to extract data
            data = run.data or {}               # Fallback to empty if no data

            # Initialize keys on the first successful extraction
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}

            # Update combined values only with non-empty results
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v

        except Exception as err:
            print(f"Skipped {fname}: {err}")  # Handle and log extraction failures

    # Proceed only if we have valid keys
    if first_keys:
        # Convert combined dictionary to a single-column DataFrame
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2023-24"])
        df.index.name = "Metric"
        
        # Sheet names in Excel are limited to 31 characters
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)

        cap = df.loc['cash_flows_from_capital_and_related_financing_activities'].iloc[0]
        noncap = df.loc['cash_flows_from_noncapital_financing_activities'].iloc[0]
        
        if pd.isna(df.loc['net_cash_from_financing_activities'].iloc[0]) or df.loc['net_cash_from_financing_activities'].iloc[0] == 0:
            # Only set if at least one component is not None/NaN
            if not pd.isna(cap) or not pd.isna(noncap):
                df.loc['net_cash_from_financing_activities'] = (cap or 0) + (noncap or 0)
        # Pull out individual cash flow components and fill missing with 0
        s_op  = df.loc['net_cash_from_operating_activities'].fillna(0)
        s_inv = df.loc['net_cash_from_investment_activities'].fillna(0)
        s_fin = df.loc['net_cash_from_financing_activities'].fillna(0)

        # Compute total net change in cash (should match reported value)
        comb = s_op + s_inv + s_fin
        orig = df.loc['change_in_cash_and_equivalents']

        # If calculated total doesn't match reported value, flag the school
        if not orig.fillna(0).equals(comb):
            test.append(school)
    else:
        print(f"No data for {school}.")

# Save the Excel file
writer.close()
print(f"All schools written to {OUTPUT_FILE}")


In [ ]:
# Set file paths for input Excel (with multiple sheets) and output Excel (with single combined sheet)
file_path   = "output_cash_flow/all_schools.xlsx"
output_path = "output_cash_flow/all_schools_combined.xlsx"

# Read all sheets from the Excel file into a dictionary of DataFrames
# Each key in `raw` is a sheet name (i.e., school), and the value is its DataFrame
raw = pd.read_excel(file_path, sheet_name=None, index_col=0)

# Create a dictionary where:
# - keys = school names
# - values = the first (and only) column of each sheet (Series of metrics)
school_series = {
    school: df.iloc[:, 0]   # Extract the only column (i.e., "2023‑24") as a Series
    for school, df in raw.items()
}

# Combine all school Series into one DataFrame:
# - Each row = one school
# - Each column = one financial metric
df_comb = pd.DataFrame(school_series).T
df_comb.index.name = "School"  # Name the row index

# Insert a "Year" column at the front for context
df_comb.insert(0, "Year", "2023‑2024")

# Write the combined DataFrame to a new Excel file with a single sheet
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_comb.to_excel(writer, sheet_name="Combined")

# Confirm that the file is saved
print("Saved:", output_path)

In [ ]:
test